Статьи с сайта https://mos.ru  
ключевые слова также со страниц со статьями (по 4 в каждой) + одно мое ключевое слово, всего для каждого текста по 5 слов

In [266]:
import nltk
import RAKE
import re
import pandas as pd
from gensim.summarization import keywords as kw
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from pymorphy2 import MorphAnalyzer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from statistics import mean 
from textdistance import levenshtein


morph = MorphAnalyzer()
nltk.download('stopwords')
stop = stopwords.words('russian')
rake = RAKE.Rake(stop)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/margaritaberseneva/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [267]:
cafe = 'туризм #ярмарка #ресторан #кафе #гид'
train = 'мцд #тариф #билет #турникет #проезд'
cable_car = 'праздник #акция #лужники #канатная дорога #день рождения'
court = 'Сергей Собянин #апелляционный суд #кассационный суд #судебный акт #суд'

In [268]:
#сбор текстов и ключевых слов

txt = {}
keys = {}

with open('1.txt', 'r', encoding='utf-8') as f:
            t = f.read()
            txt ['cafe'] = t
            keys ['cafe'] = cafe
            
with open('2.txt', 'r', encoding='utf-8') as f:
            t = f.read()
            txt ['train'] = t
            keys ['train'] = train

with open('3.txt', 'r', encoding='utf-8') as f:
            t = f.read()
            txt ['cable_car'] = t
            keys ['cable_car'] = cable_car

with open('4.txt', 'r', encoding='utf-8') as f:
            t = f.read()
            txt ['court'] = t
            keys ['court'] = court

In [269]:
#лемматизация текстов

new_txt = {}

for t in txt.keys():
    text = txt[t]
    text = re.sub(r'\n', r' ', text)
    tokenizer = RegexpTokenizer(r'\w+')
    text = tokenizer.tokenize(text)
    new = ''
    for w in text:
        l = morph.parse(w)[0].normal_form + ' '
        new += l
    new_txt[t] = new

In [270]:
#пробуем пройтись с ключевыми словами по лемматизированным текстам 

for key, value in keys.items():
    v = value.split(' #')
    for w, text in new_txt.items():
        if w == key:
            print(key)
            for word in v:
                c = new_txt[w].count(word)
                print(word, ' — ', c)
            print('\n')
            keys[key] = v

cafe
туризм  —  1
ярмарка  —  0
ресторан  —  5
кафе  —  0
гид  —  3


train
мцд  —  17
тариф  —  6
билет  —  1
турникет  —  3
проезд  —  6


cable_car
праздник  —  0
акция  —  0
лужники  —  4
канатная дорога  —  0
день рождения  —  0


court
Сергей Собянин  —  0
апелляционный суд  —  3
кассационный суд  —  4
судебный акт  —  4
суд  —  25




Более-менее что-то вырисовывается. Ключевые слова в большинстве своем встречаются в текстах. Субъективно лучше всего с этим обстоят дела у текста про поезда, затем у текстов про суды и кафе. Совсем все плохо у текста про канатную дорогу, но все равно есть хоть что-то.

# Rake

In [278]:
ake_txt = {}

for key in new_txt.keys():
    print(key)
    rake_keys_list = rake.run(new_txt[key], maxWords=2, minFrequency=1)
    keys_ = []
    for k in rake_keys_list:
        print(k)
        keys_.append(k[0])
    rake_txt[key] = keys_
    print('\n')

cafe
('редакция гид', 4.0)
('китайский язык', 4.0)
('поддержка комитет', 4.0)
('иностранный язык', 4.0)
('сборник попасть', 3.5)
('сборник', 1.5)
('путеводитель', 1.0)
('русский', 1.0)
('английский', 1.0)
('войти', 1.0)
('сервис', 1.0)


train
('тройка стрелка', 4.0)
('это сообщить', 4.0)
('карта движение', 4.0)
('первое диаметр', 4.0)
('станция марк', 4.0)
('который поездка', 4.0)
('это пересадка', 4.0)
('оплата проезд', 3.75)
('остафьево проезд', 3.75)
('проезд оплачиваться', 3.75)
('сутки ожидаться', 3.5)
('год тариф', 3.5)
('проезд', 1.75)
('сутки', 1.5)
('год', 1.5)
('сегодня', 1.0)
('валидатор', 1.0)
('понедельник', 1.0)
('25 ноябрь', 1.0)
('мцд 1', 1.0)
('миллион', 1.0)
('совершить', 1.0)
('второе', 1.0)
('пассажиропоток', 1.0)
('связь', 1.0)
('мцд', 1.0)
('сетунь', 1.0)
('мцд 2', 1.0)
('метро', 1.0)
('предел', 1.0)
('мцк', 1.0)
('10', 0)
('11', 0)


cable_car
('канатный дорога', 4.0)
('весь желающий', 4.0)
('воробьев гора', 4.0)
('удобный возможность', 4.0)
('пять минута', 4.0)

In [279]:
for k, v in keys.items():
    print(k)
    keys_real = keys[k]
    print (keys_real)
    my_keys = rake_txt[k]
    print (my_keys)
    pred = keys_real
    real = my_keys
    tp = 0
    fn = 0
    fp = 0
    for p in pred:
        if p in real:
            tp += 1
        if p not in real:
            fp += 1
    for i in real:
        if i not in pred:
            fn += 1
    accuracy = tp / (tp + fn + fp)
    print('accuracy: ', accuracy)
    precision = tp / (tp + fp)
    print('precision: ', precision)
    try:
        recall = tp / (tp + fn)
    except ZeroDivisionError:
        recall = 0.0    
    try:
        f1 = (2 * precision * recall) / (precision + recall)
    except ZeroDivisionError:
        f1 = 0.0
    print('f1: ', f1)
    print('\n')

cafe
['туризм', 'ярмарка', 'ресторан', 'кафе', 'гид']
['редакция гид', 'китайский язык', 'поддержка комитет', 'иностранный язык', 'сборник попасть', 'сборник', 'путеводитель', 'русский', 'английский', 'войти', 'сервис']
accuracy:  0.0
precision:  0.0
f1:  0.0


train
['мцд', 'тариф', 'билет', 'турникет', 'проезд']
['тройка стрелка', 'это сообщить', 'карта движение', 'первое диаметр', 'станция марк', 'который поездка', 'это пересадка', 'оплата проезд', 'остафьево проезд', 'проезд оплачиваться', 'сутки ожидаться', 'год тариф', 'проезд', 'сутки', 'год', 'сегодня', 'валидатор', 'понедельник', '25 ноябрь', 'мцд 1', 'миллион', 'совершить', 'второе', 'пассажиропоток', 'связь', 'мцд', 'сетунь', 'мцд 2', 'метро', 'предел', 'мцк', '10', '11']
accuracy:  0.05555555555555555
precision:  0.4
f1:  0.10526315789473685


cable_car
['праздник', 'акция', 'лужники', 'канатная дорога', 'день рождения']
['канатный дорога', 'весь желающий', 'воробьев гора', 'удобный возможность', 'пять минута', 'велосипед л

С RAKE как-то вышло слабо, но результат тем не менее есть. Он хорошо выделил ключевые словосочетания и слова, по ним действительно можно понять о чем текст. Но все словосочетания никак не совспадают с заранее выделенными ключевыми словами, совпадения только у выделенных RAKEом слов, и то совсем у небольшого количества.

# Textrank

In [273]:
textrank_txt = {}

for key in new_txt.keys():
    print(key)
    textrank = kw(new_txt[key], pos_filter=[], scores=True)
    keys_ = []
    for k in textrank[:5]:
        keys_.append(k[0])
        print(k[0], k[1])
    textrank_txt[key] = keys_
    print('\n')

cafe
ресторан 0.22023828851792784
гид 0.2137077768574432
заведение столица 0.20524204491381853
издание 0.2038979419676054
ресторанныи 0.1874563370088292


train
мцд 0.4463949642279051
станция 0.21673025800480625
это 0.19837396638301416
миллион 0.19440575437152344
проезд 0.19418597260437276


cable_car
для 0.3697738868221354
новыи 0.1983960711642825
канатныи дорога 0.17688651564207897
лужники 0.1551839094382837
минута 0.14794253321944487


court
строительство 0.23634906298475725
судно 0.22776385308309677
здание 0.22398178475367836
год 0.20088151666988502
кассационныи суд 0.1907985063696922




In [274]:
for k, v in keys.items():
    print(k)
    keys_real = []
    for w in keys[k]:
        if 'й' in w:
            w = w.replace('й', 'и')
        keys_real.append(w)
    print (keys_real)
    my_keys = textrank_txt[k]
    print (my_keys)
    pred = keys_real
    real = my_keys
    tp = 0
    fn = 0
    fp = 0
    for p in pred:
        if p in real:
            tp += 1
        if p not in real:
            fp += 1
    for i in real:
        if i not in pred:
            fn += 1
    accuracy = tp / (tp + fn + fp)
    print('accuracy: ', accuracy)
    precision = tp / (tp + fp)
    print('precision: ', precision)
    try:
        recall = tp / (tp + fn)
    except ZeroDivisionError:
        recall = 0.0    
    try:
        f1 = (2 * precision * recall) / (precision + recall)
    except ZeroDivisionError:
        f1 = 0.0
    print('f1: ', f1)
    print('\n')

cafe
['туризм', 'ярмарка', 'ресторан', 'кафе', 'гид']
['ресторан', 'гид', 'заведение столица', 'издание', 'ресторанныи']
accuracy:  0.25
precision:  0.4
f1:  0.4000000000000001


train
['мцд', 'тариф', 'билет', 'турникет', 'проезд']
['мцд', 'станция', 'это', 'миллион', 'проезд']
accuracy:  0.25
precision:  0.4
f1:  0.4000000000000001


cable_car
['праздник', 'акция', 'лужники', 'канатная дорога', 'день рождения']
['для', 'новыи', 'канатныи дорога', 'лужники', 'минута']
accuracy:  0.1111111111111111
precision:  0.2
f1:  0.20000000000000004


court
['Сергеи Собянин', 'апелляционныи суд', 'кассационныи суд', 'судебныи акт', 'суд']
['строительство', 'судно', 'здание', 'год', 'кассационныи суд']
accuracy:  0.1111111111111111
precision:  0.2
f1:  0.20000000000000004




У Textrank дела обстоят заметно лучше. Он уже орудует больше словами, чем словосочетаниями, а способность понять о чем был текст по выделенным словам не утрачивает. Здесь и метрики лучше сработали. У текстов про кафе и поезда лучше всего. Далее тексты про канатную дорогу и про суды. Занятно, что у Textrank куда-то делась буква 'й'. Вместо нее он встявляет в слова 'и'. Для этого я тоже немного отредактировала свои слова, чтобы метрики были получше.

# TF-IDF

In [275]:
corpus = list(new_txt.values())
cv=CountVectorizer(max_features=10000)
word_count_vector=cv.fit_transform(corpus)
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tf_idf_vector=tfidf_transformer.fit_transform(word_count_vector)
df = pd.DataFrame(tf_idf_vector.toarray(), columns=cv.get_feature_names(), index=txt.keys())
stop = set(stop)
words = set(df.columns)
stop_words = stop.intersection(words)
for w in stop_words:
    df = df.drop([w], axis=1)
df = df.T

In [276]:
tfidf_txt = {}

names = list(df.columns)
for c in names:
    k = df.nlargest(5, c)
    k = k[c]
    print(c)
    print(k)
    tfidf_txt[c] = list(k.index)
    print('\n')

cafe
заведение    0.321130
столица      0.253182
гид          0.240847
ресторан     0.240847
язык         0.189887
Name: cafe, dtype: float64


train
мцд        0.596835
поездка    0.210648
рубль      0.175540
миллион    0.166077
проезд     0.166077
Name: train, dtype: float64


cable_car
воробьев    0.240069
гора        0.240069
канатный    0.240069
лужники     0.240069
дорога      0.236592
Name: cable_car, dtype: float64


court
судно            0.357545
суд              0.321791
строительство    0.250282
апелляционный    0.214527
здание           0.214527
Name: court, dtype: float64




In [277]:
for k, v in keys.items():
    print(k)
    keys_real = keys[k]
    print (keys_real)
    my_keys = tfidf_txt[k]
    print (my_keys)
    pred = keys_real
    real = my_keys
    tp = 0
    fn = 0
    fp = 0
    for p in pred:
        if p in real:
            tp += 1
        if p not in real:
            fp += 1
    for i in real:
        if i not in pred:
            fn += 1
    accuracy = tp / (tp + fn + fp)
    print('accuracy: ', accuracy)
    precision = tp / (tp + fp)
    print('precision: ', precision)
    try:
        recall = tp / (tp + fn)
    except ZeroDivisionError:
        recall = 0.0    
    try:
        f1 = (2 * precision * recall) / (precision + recall)
    except ZeroDivisionError:
        f1 = 0.0
    print('f1: ', f1)
    print('\n')

cafe
['туризм', 'ярмарка', 'ресторан', 'кафе', 'гид']
['заведение', 'столица', 'гид', 'ресторан', 'язык']
accuracy:  0.25
precision:  0.4
f1:  0.4000000000000001


train
['мцд', 'тариф', 'билет', 'турникет', 'проезд']
['мцд', 'поездка', 'рубль', 'миллион', 'проезд']
accuracy:  0.25
precision:  0.4
f1:  0.4000000000000001


cable_car
['праздник', 'акция', 'лужники', 'канатная дорога', 'день рождения']
['воробьев', 'гора', 'канатный', 'лужники', 'дорога']
accuracy:  0.1111111111111111
precision:  0.2
f1:  0.20000000000000004


court
['Сергей Собянин', 'апелляционный суд', 'кассационный суд', 'судебный акт', 'суд']
['судно', 'суд', 'строительство', 'апелляционный', 'здание']
accuracy:  0.1111111111111111
precision:  0.2
f1:  0.20000000000000004




У TF-IDF ситуация еще лучше, чем у двух предыдущих методов. По выделенным словам все так же можно понять о чем был текст. Метрики здесь показывают лучшие значения по сравнению с другими методами (но плохие, если смотреть на числа). Лидеры — тексты про кафе и поезда. Два остальных текста показывают меньшие значения по метрикам.

Попробуем представить полученные результаты для разных методов и текстов одной таблице, где 4 балла -- лучший результат, 1 -- худший.

|| Cafe  | Train | Cable car | Court |
|-------------| ------------- | ------------- | ------------- | ------------- |
| Ключевые слова | 2 | 4 | 1 | 3 |
| RAKE | 0 | 2 | 1 | 0 |
| Textrank | 4 | 4 | 2 | 2 |
| TF-IDF | 4 | 4 | 2 | 2 |
| ------------- | ------------- | ------------- | ------------- | ------------- |
| Итог | 10 | 14 | 8 | 7 |

Методы Textrank и TF-IDF оказались хороши. RAKE выделяет словосочетания, для нашей цели это оказалась отнюдь не преимуществом. По результатам подсчетов победу одержал текст про поезда.  
Из очевидных трудностей могу назвать такой кейс: канатная дорога после лемматизации становится "канатный дорога". В таком виде она уже не сможет ни с чем совпасть. Также выделение словосочетаний как ключевых слов не лучшая идея. Самое ходовое решение -- существительное в единственном числе (в крайнем случае можно прилагательное).